# Partie 1 - Construction du CNN

In [2]:
# Importation des modules
from keras.models import Sequential     # Initialiser le réseau de neurones
from keras.layers import Convolution2D  # Etape de convulution (image en 2D)
from keras.layers import MaxPooling2D   # Etape de MaxPooling (image en 2D)
from keras.layers import Flatten        # Applatir toutes les features map dans un vecteur vertical
from keras.layers import Dense          # Ajouter des couches complètements connectées

In [3]:
# Initialise le CNN
classifier = Sequential()

In [5]:
# Etape 1 : Couche de convolution
classifier.add(Convolution2D(filters=32,               # 32 features maps
                             kernel_size=3,            # Feature detector size = 3x3
                             strides=1,                # On décalle le feature detector de 1 pixel à chaque fois
                             input_shape =(64, 64, 3), # taille 64x64, 3 car images en couleurs
                             activation = "relu"))     # fonction redresseur, on laisse passer le signal ou non

In [6]:
# Etape 2 : Couche de pooling
classifier.add(MaxPooling2D(pool_size=(2,2))) # On divise la taille de chaque feature map par 2

(# Etape 2.5 : Couche de convolution et pooling supplémentaire pour augmenter l'accuracy)
classifier.add(Convolution2D(filters=32,               
                             kernel_size=3,            
                             strides=1,                
                             activation = "relu"))     

classifier.add(MaxPooling2D(pool_size=(2,2)))

In [7]:
# Etape 3 : Couche de flattening
classifier.add(Flatten()) # On applatit toutes les features map dans un vecteur vertical

In [8]:
# Etape 4 : Couche complètement connectée
classifier.add(Dense(units=128, activation="relu")) # 128 neurones + relu
classifier.add(Dense(units=1,   activation="sigmoid")) # 1 neurone de sortie + sigmoid pour avoir la probabilité de la classification

In [10]:
# Compilation du CNN
classifier.compile(optimizer="adam",           # Algorithme du gradient
                   loss="binary_crossentropy", # Fonction de coût pour faire de la classification avec 2 catégories
                   metrics=["accuracy"])      # Mesurer la performance

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Entraîner le CNN sur nos images
# Attention à l'overfitting

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2, # Transvection
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,         # 8000 images répartis dans 32 lots différents
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,              # 2000 images répartis dans 32 lots différents
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [13]:
# Entraînement
classifier.fit_generator(training_set,
                        steps_per_epoch=250,        # 250 images de training dans chaque lot (250*32 = 8000 images)
                        epochs=25,                  # On passe 25 fois sur le training set
                        validation_data=test_set,
                        validation_steps=63)        # 63 images de test dans chaque lot (62.5*32 = 2000 images)


Epoch 1/25
250/250 [==============================] - 150s 601ms/step - loss: 0.6955 - accuracy: 0.6294 - val_loss: 0.4663 - val_accuracy: 0.6555
Epoch 2/25
250/250 [==============================] - 33s 131ms/step - loss: 0.5900 - accuracy: 0.6913 - val_loss: 0.4510 - val_accuracy: 0.6930
Epoch 3/25
250/250 [==============================] - 34s 137ms/step - loss: 0.5559 - accuracy: 0.7140 - val_loss: 0.8357 - val_accuracy: 0.7230
Epoch 4/25
250/250 [==============================] - 33s 131ms/step - loss: 0.5390 - accuracy: 0.7312 - val_loss: 0.5765 - val_accuracy: 0.6900
Epoch 5/25
250/250 [==============================] - 33s 132ms/step - loss: 0.5204 - accuracy: 0.7380 - val_loss: 0.5359 - val_accuracy: 0.7415
Epoch 6/25
250/250 [==============================] - 32s 126ms/step - loss: 0.5133 - accuracy: 0.7435 - val_loss: 0.6429 - val_accuracy: 0.7415
Epoch 7/25
250/250 [==============================] - 33s 134ms/step - loss: 0.4950 - accuracy: 0.7563 - val_loss: 0.4731 - val_

In [36]:
# Single prediction

import numpy as np
from keras.preprocessing import image
test_image = image.load_img("dataset/single_prediction/cat_or_dog_1.jpg", target_size = (64, 64))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)
training_set.class_indices # format

if result[0][0] == 1: # Si première ligne première colonne == 1
    prediction = "Je pense que c'est un chien !"
else:
    prediction = "Je pense que c'est un chat !"
print(prediction)

Je pense que c'est un chien !
